In [1]:
import gzip
import pickle as pkl
import gc

import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score


In [4]:
DATA_PATH = "../data/"
MODEL_ORIG_PATH = "../models_original/"
MODEL_PATH = "../models/"


In [3]:
DATA_NPZ_FILE = f'{DATA_PATH}dataTrainValTest.npz'


print ('reading from saved file Data.npz: ', DATA_NPZ_FILE)
data = np.load(DATA_NPZ_FILE)
print('data file contents: ', data.files)

x_test = data['x_test']
cats_test = data['cats_test']
reg_y_test = data['reg_y_test']
rol_y_test = data['rol_y_test']

print('x_test.shape:      ', x_test.shape)
print('cats_test.shape:   ', cats_test.shape)
print('reg_y_test.shape:  ', reg_y_test.shape)
print('rol_y_test.shape:  ', rol_y_test.shape)


reading from saved file Data.npz:  ../data/dataTrainValTest.npz
data file contents:  ['x_train', 'x_val', 'x_test', 'cats_train', 'cats_val', 'cats_test', 'reg_y_train', 'reg_y_val', 'reg_y_test', 'rol_y_train', 'rol_y_val', 'rol_y_test']
x_test.shape:       (79925, 2199)
cats_test.shape:    (79925, 15)
reg_y_test.shape:   (79925, 4103)
rol_y_test.shape:   (79925, 781)


In [6]:
def evaluate_model(y, yhat):
    preds = np.around(yhat).astype(np.int32)
    p = round(precision_score(y, preds, average='micro') * 100, 2)
    r = round(recall_score(y, preds, average='micro') * 100, 2)
    f1 = round(f1_score(y, preds, average='micro') * 100, 2)
    return p, r, f1

In [5]:
# Load trained models with NO adjustment of initial weights

model_orig_bot_rolled = load_model(f'{MODEL_ORIG_PATH}BOTModelOriginal.h5')
model_orig_bot_regular = load_model(f'{MODEL_ORIG_PATH}BOTModelRegular.h5')
model_orig_cnn3_rolled = load_model(f'{MODEL_ORIG_PATH}CNN3LayerOriginal.h5')
model_orig_cnn3_regular = load_model(f'{MODEL_ORIG_PATH}CNN3LayerOriginalRegularPartialSet.h5')
model_orig_cnnbaseline_rolled = load_model(f'{MODEL_ORIG_PATH}CNNBaselineOriginal.h5')
model_orig_cnnbaseline_regular = load_model(f'{MODEL_ORIG_PATH}CNNModelRegularPartialDataset.h5')

2022-05-04 23:09:24.307356: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# predict for test dataset using various models
reg_orig_yhat_bot = model_orig_bot_regular.predict(x_test)
print('reg_orig_yhat_bot done... reg_orig_yhat_bot.shape: ', reg_orig_yhat_bot.shape)
reg_orig_yhat_cnn = model_orig_cnnbaseline_regular.predict(x_test)
print('reg_orig_yhat_cnn done..., reg_orig_yhat_cnn.shape: ', reg_orig_yhat_cnn.shape)
reg_orig_yhat_cnn3 = model_orig_cnn3_regular.predict(x_test)
print('reg_orig_yhat_cnn3 done..., reg_orig_yhat_cnn3.shape: ', reg_orig_yhat_cnn3.shape)


rol_orig_yhat_bot = model_orig_bot_rolled.predict(x_test)
print('rol_orig_yhat_bot done..., rol_orig_yhat_bot.shape: ', rol_orig_yhat_bot.shape)

rol_orig_yhat_cnn = model_orig_cnnbaseline_rolled.predict(x_test)
print('rol_orig_yhat_cnn done..., rol_orig_yhat_cnn.shape:', rol_orig_yhat_cnn.shape)

rol_orig_yhat_cnn3 = model_orig_cnn3_rolled.predict(x_test)
print('rol_orig_yhat_cnn3 done..., rol_yhat_cnn3.shape: ', rol_orig_yhat_cnn3.shape)

reg_orig_yhat_bot done... reg_orig_yhat_bot.shape:  (79925, 4103)
reg_orig_yhat_cnn done..., reg_orig_yhat_cnn.shape:  (79925, 4103)
reg_orig_yhat_cnn3 done..., reg_orig_yhat_cnn3.shape:  (79925, 4103)
rol_orig_yhat_bot done..., rol_orig_yhat_bot.shape:  (79925, 781)
rol_orig_yhat_cnn done..., rol_orig_yhat_cnn.shape: (79925, 781)
rol_orig_yhat_cnn3 done..., rol_yhat_cnn3.shape:  (79925, 781)


In [10]:
reg_orig_bot_result = evaluate_model(reg_y_test, reg_orig_yhat_bot)
print('BoT Regular (precision, recall, F1)         : ', reg_orig_bot_result)
reg_orig_cnn_result = evaluate_model(reg_y_test, reg_orig_yhat_cnn)
print('CNN Regular (precision, recall, F1)        : ', reg_orig_cnn_result)
reg_orig_cnn3_result = evaluate_model(reg_y_test, reg_orig_yhat_cnn3)
print('CNN3 Regular (precision, recall, F1)       : ', reg_orig_cnn3_result)
rol_orig_bot_result = evaluate_model(rol_y_test, rol_orig_yhat_bot)
print('BoT Rolled (precision, recall, F1)         : ', rol_orig_bot_result)
rol_orig_cnn_result = evaluate_model(rol_y_test, rol_orig_yhat_cnn)
print('CNN Rolled (precision, recall, F1)         : ', rol_orig_cnn_result)
rol_orig_cnn3_result = evaluate_model(rol_y_test, rol_orig_yhat_cnn3)
print('CNN3 Rolled (precision, recall, F1)        : ', rol_orig_cnn3_result)

BoT Regular (precision, recall, F1)         :  (63.79, 5.04, 9.34)
CNN Regular (precision, recall, F1)        :  (72.03, 15.82, 25.94)
CNN3 Regular (precision, recall, F1)       :  (75.82, 30.71, 43.71)
BoT Rolled (precision, recall, F1)         :  (85.67, 10.32, 18.43)
CNN Rolled (precision, recall, F1)         :  (77.51, 28.68, 41.87)
CNN3 Rolled (precision, recall, F1)        :  (79.56, 38.33, 51.73)


In [25]:
# Load trained models with adjustment of initial weights

model_bot_rolled = load_model(f'{MODEL_PATH}BOTModel.h5')
# model_bot_regular = load_model(f'{MODEL_PATH}BOTModelRegular.h5')
model_cnn3_rolled = load_model(f'{MODEL_PATH}CNN3LayerV2.h5')
model_cnn3_regular = load_model(f'{MODEL_PATH}CNN3LayerRegularSet4.h5')
model_cnnbaseline_rolled = load_model(f'{MODEL_PATH}CNNBaseline.h5')
model_cnnbaseline_regular = load_model(f'{MODEL_PATH}CNNBaselineRegularSet4.h5')


In [27]:
# predict for test dataset using various models
# reg_yhat_bot = model_bot_regular.predict(x_test)
print('reg_yhat_bot done...')
reg_yhat_cnn = model_cnnbaseline_regular.predict(x_test)
print('reg_yhat_cnn done..., reg_yhat_cnn.shape: ', reg_yhat_cnn.shape)
reg_yhat_cnn3 = model_cnn3_regular.predict(x_test)
print('reg_yhat_cnn done..., reg_yhat_cnn3.shape: ', reg_yhat_cnn3.shape)


rol_yhat_bot = model_bot_rolled.predict(x_test)
print('rol_yhat_bot done..., rol_yhat_bot.shape: ', rol_yhat_bot.shape)

rol_yhat_cnn = model_cnnbaseline_rolled.predict(x_test)
print('rol_yhat_cnn done..., rol_yhat_cnn.shape:', rol_yhat_cnn.shape)

rol_yhat_cnn3 = model_cnn3_rolled.predict(x_test)
print('rol_yhat_cnn3 done..., rol_yhat_cnn3.shape: ', rol_yhat_cnn3.shape)


reg_yhat_bot done...
reg_yhat_cnn done..., reg_yhat_cnn.shape:  (79925, 4103)
reg_yhat_cnn done..., reg_yhat_cnn3.shape:  (79925, 4103)
rol_yhat_bot done..., rol_yhat_bot.shape:  (79925, 781)
rol_yhat_cnn done..., rol_yhat_cnn.shape: (79925, 781)
rol_yhat_cnn3 done..., rol_yhat_cnn3.shape:  (79925, 781)


In [36]:

reg_cnn_result = evaluate_model(reg_y_test, reg_yhat_cnn)
print('CNN Regular (precision, recall, F1)        : ', reg_cnn_result)
reg_cnn3_result = evaluate_model(reg_y_test, reg_yhat_cnn3)
print('CNN3 Regular (precision, recall, F1)       : ', reg_cnn3_result)
rol_bot_result = evaluate_model(rol_y_test, rol_yhat_bot)
print('BoT Rolled (precision, recall, F1)         : ', rol_bot_result)
rol_cnn_result = evaluate_model(rol_y_test, rol_yhat_cnn)
print('CNN Rolled (precision, recall, F1)         : ', rol_cnn_result)
rol_cnn3_result = evaluate_model(rol_y_test, rol_yhat_cnn3)
print('CNN3 Rolled (precision, recall, F1)        : ', rol_cnn3_result)


CNN Regular (precision, recall, F1)        :  (74.98, 21.91, 33.91)
CNN3 Regular (precision, recall, F1)       :  (75.95, 32.44, 45.46)
BoT Rolled (precision, recall, F1)         :  (78.58, 31.81, 45.29)
CNN Rolled (precision, recall, F1)         :  (79.14, 44.66, 57.1)
CNN3 Rolled (precision, recall, F1)        :  (81.13, 43.56, 56.68)
